# Python для Анализа данных



## Домашнее задание 10: Pipeline описательного анализа данных

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
%matplotlib inline

In [ ]:
sns.set_style('darkgrid')

In [ ]:
happy = pd.read_csv('../input/world-happiness-report-2019/world-happiness-report-2019.csv')
happy.head()

In [ ]:
# Для дальнейшего удобства переименную столбцы
happy = happy.rename( columns = {'Country (region)': 'Country', 'SD of Ladder': 'SD', 
                                 'Positive affect': 'Positive', 'Negative affect':'Negative',
                                 'Social support':'Social', 'Log of GDP\nper capita': 'l_GPD',
                                 'Healthy life\nexpectancy': 'HLE'
                                } )
happy.head()

In [ ]:
happy.shape

Country (region)
Name of the country.

Ladder
Cantril Ladder is a measure of life satisfaction.

SD of Ladder
Standard deviation of the ladder.

Positive affect
Measure of positive emotion.

Negative affect
Measure of negative emotion.

Social support
The extent to which Social support contributed to the calculation of the Happiness Score.

Freedom
The extent to which Freedom contributed to the calculation of the Happiness Score.

Corruption
The extent to which Perception of Corruption contributes to Happiness Score.

Generosity
The extent to which Generosity contributed to the calculation of the Happiness Score.

Log of GDP per capita
The extent to which GDP contributes to the calculation of the Happiness Score.

Healthy life expectancy
The extent to which Life expectancy contributed to the calculation of the Happiness Score.


Целевая переменной должна быть: Ladder. Мера удовлетворения жизнью.
Но так как значения в ней соответствуют индексу/количеству стран. 1-155, её в расчёт не берём.

Целевой переменной выбираем Positive (Positive affect). Мера положительных эмоций.

In [ ]:
#Проверка на пропусков
print(np.round(happy.isna().sum()[happy.isna().sum()>0] / happy.shape[0], 2)) # В процентах
print('-------------')
print(happy.isna().sum()) # По количеству пропусков

In [ ]:
happy.dropna().shape[0]/happy.shape[0]


#Если дропнуть все строки с NaN останется 89,8% данных.
#Да, с ними можно работать.
#Но, так как я хочу увидеть полную информацию по всем странам, то буду заполнять данные.


In [ ]:
# Проверка корреляции столбцов с nan значениями с целевой переменной Ladder
# Так же проверка на корреляцию при замене пропусков на квантиль 0,45

for i in happy.isna().sum()[happy.isna().sum().values > 0].index.tolist():
    print( happy[i].corr(happy['Positive']),' :', i )
    print( happy[i].fillna(happy[i].quantile(0.45)).corr(happy['Positive']) )
    
# Получившиеся данные меня устраивают

In [ ]:
for i in happy.isna().sum()[happy.isna().sum().values > 0].index.tolist():
    happy[i].fillna(happy[i].quantile(0.45), inplace=True)
happy.isna().sum()

In [ ]:
#Столбец Ladder не информативен. Дропаем.
happy.drop('Ladder', axis=1, inplace=True)

In [ ]:
plt.figure( figsize=(12,10) )
sns.heatmap( happy.corr() , cmap='coolwarm', vmin=-1, center=0, annot=True, fmt='.2f')
plt.show()

In [ ]:
posit_bins = happy.Positive.quantile([0, 0.25, 0.5, 0.75, 1])
posit_bins

In [ ]:
happy['Positive_Group'] = pd.cut(happy['Positive'], posit_bins, \
                                labels=['0-0.25', '0.25-0.5', '0.5-0.75','0.75-1'], \
                                right=True, include_lowest=True)

In [ ]:
happy.head()

In [ ]:
cols = ['Positive','Freedom', 'Social', 'Generosity', 'HLE']

In [ ]:
cols.append('Positive_Group')

In [ ]:
sns.pairplot(happy[cols], hue='Positive_Group')
plt.show()

In [ ]:
#Распределение признаков
fig = plt.figure(figsize=(13,25))

ax1 = fig.add_subplot(421)
ax2 = fig.add_subplot(422)
ax3 = fig.add_subplot(423)
ax4 = fig.add_subplot(424)

a=sns.violinplot(data=happy, x='Freedom', y='Positive_Group', hue='Positive_Group', ax=ax1)
a.set_title('Корреляция 0,68')
b=sns.violinplot(data=happy, x='Generosity', y='Positive_Group', hue='Positive_Group', ax=ax2)
b.set_title('Корреляция 0,39')
c=sns.violinplot(data=happy, x='Social', y='Positive_Group', hue='Positive_Group', ax=ax3)
c.set_title('Корреляция 0,32')
d=sns.violinplot(data=happy, x='HLE', y='Positive_Group', hue='Positive_Group', ax=ax4)
d.set_title('Корреляция 0,32')

plt.show()

In [ ]:
plt.figure(figsize=(30,25))

sns.lmplot(data=happy, x='Freedom', y='Social', hue='Positive_Group')
sns.lmplot(data=happy, x='Freedom', y='Generosity', hue='Positive_Group')
sns.lmplot(data=happy, x='Freedom', y='HLE', hue='Positive_Group')


plt.show()

### Краткое описание:

Целевым признаком выбран Positive (Мера положительных эмоций).
Параметр разбил по группам квантилей 0-25, 25-50, 50-75, 75-100.

Исходя из матрицы корреляций положительную корреляцию имеют признаки:
Freedom - 0.68
Social - 0.39
Negative - 0.39
Generosity - 0.35
HLE - 0.32
I_GPD - 0.29
SD - 0.09

Так как признак Negative противопоставляется целевой переменной, его в расчёт не беру. I_GPD, SD так же выбрасываю.

#### На основе графиков распределения:
1) С увеличением Freedom (свободы), явно увеличивается количество счастливых людей.

2) С увеличением Generosity (щедрости), явно увеличивается количество счастливых людей.

3) С увеличением Social (соц. поддержки), увеличивается количество счастливых людей.

4) С увеличением HLE (ожидаемой продолжительности жизни), неявно увеличивается количество счастливых людей.

#### На основе графиков парных взаимосвязей Свободы с разбивкой по группам счастья:
1) Social - Явная положительная зависимость.

2) Generosity - Явная положительная зависимость.

3) HLE - Явная положительная зависимость.

## На основе анализа:
Чтобы повысить уровень счастья населения (Positive) необходимо увеличить: Freedom, Social, Generosity.

### Задача 2

Возьмите любой датасет **для задачи классификации** (с работы, с учебы, из репозиториев) и проведите описательный анализ данных. Сделайте краткий вывод о найденных взаимосвязях.

In [ ]:
pok = pd.read_csv('../input/pokemon/Pokemon.csv', index_col=0)
pok.head(10)

In [ ]:
pok.shape

In [ ]:
print(pok.isna().sum()[pok.isna().sum()>0])
print(np.round(pok.isna().sum()[pok.isna().sum()>0] / pok.shape[0], 2))

In [ ]:
pok.drop('Type 2', axis=1, inplace=True)

In [ ]:
pok['Type 1'].unique()

In [ ]:
pok['Type 1'].nunique()

In [ ]:
# Деление типов на архитипы для удобного анализа

d = dict()

for i in range(6):
  a = pok['Type 1'].unique()[i::6]
  for j in a:
    d[j] = i
print(d)

In [ ]:
#Создаём столбец и заполняем 
pok['Class'] = pok['Type 1']
pok['Class'] = pok['Class'].replace(d)

In [ ]:
pok['Type 1'].unique()

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap( pok.corr(), cmap='coolwarm', vmin=-1, center=0, annot=True )
plt.show()

In [ ]:
total_bins = pok.Total.quantile([0, 0.25, 0.5, 0.75, 1])
pok['Total_Groups'] = pd.cut(pok['Total'], total_bins, \
                                labels=['0-0.25', '0.25-0.5', '0.5-0.75','0.75-1'], \
                                right=True, include_lowest=True)
pok.head()

In [ ]:
#Из графика видно,что концентрация группы 0,75-1 выше в классах 2,4
plt.figure(figsize=(10,8))
sns.violinplot(data=pok, x='Total_Groups', y='Class', hue='Total_Groups')
plt.show()

In [ ]:
sns.distplot(pok.Class)

In [ ]:
top_pok = pok[ (pok.Class == 2) | (pok.Class == 4) ]
top_pok = top_pok [ top_pok.Total_Groups == '0.75-1' ]
top_pok.head()

In [ ]:
total_bins = top_pok.Total.quantile([0, 0.25, 0.5, 0.75, 1])
top_pok['Total_Groups'] = pd.cut(top_pok['Total'], total_bins, \
                                labels=['0-0.25', '0.25-0.5', '0.5-0.75','0.75-1'], \
                                right=True, include_lowest=True)
top_pok.head()

In [ ]:
 #top_pok.sort_values(by = ['Total'], ascending = False)

d = dict()

for i in range(6):
  a = top_pok['Type 1'].unique()[i::1]
  for j in a:
    d[j] = i
print(d)

top_pok['Class'] = top_pok['Type 1']
top_pok['Class'] = top_pok['Class'].replace(d)


In [ ]:
top_pok.head()

In [ ]:
# По графику: Высокая концентрация 0,75-1 у класса №3. Затем идут классы 2 (привалирует 0,25-0,5 над 0,5-0,75) и 1 (низкие 0,75-1 и 0,25-0,5, средние 0,5-0,75)
plt.figure(figsize=(10,8))
sns.violinplot(data=top_pok, y='Total_Groups', x='Class', hue='Total_Groups')
plt.show()

In [ ]:
#Оставляем топ классы
top_pok = top_pok[ (top_pok.Class == 3) | (top_pok.Class == 2) | (top_pok.Class == 1)  ]

In [ ]:
#Появился коэф корреляции переменной Class с Total, Attack, Defence, Generation, Legendary
plt.figure(figsize=(10,8))
sns.heatmap( top_pok.corr(), cmap='coolwarm', vmin=-1, center=0, annot=True )
plt.show()

In [ ]:
sns.lmplot(data=top_pok, y='Legendary', x='Class', hue='Total_Groups', logx=True)

In [ ]:
sns.lmplot(data=top_pok, y='Attack', x='Class', hue='Total_Groups')

In [ ]:
sns.lmplot(data=top_pok, y='Total', x='Class', hue='Total_Groups')

In [ ]:
Top_10_Dragon = top_pok[ top_pok.Class == 3  ].sort_values(by = ['Total'], ascending = False)[:10].drop(['Class','Total_Groups'], axis=1)

In [ ]:
Top_10_Dragon.head()

In [ ]:
Top_10_Psychic = top_pok[ top_pok.Class == 2  ].sort_values(by = ['Total'], ascending = False)[:10].drop(['Class','Total_Groups'], axis=1)

In [ ]:
Top_10_Psychic.head()

### Краткое описание:

Целевым признаком выбран Type 1 (Тип покемона) и Total (Суммарное количество характеристик)
Так как хотел выявить зависимость показателей от Type 1, пришлось заменить через словарь значения.
Уникальных значений в столбце Type 1 = 18. Данных показателей слишком много. Решено было разбить их на 3 уберКласса по 6 подклассов и работать с ними.

ПараметрTotal разбил по группам квантилей 0-25, 25-50, 50-75, 75-100.

#### Матрица корреляции по 3 уберКлассам не информативна.

#### Работа с графиком распределения:
Из графикв видно,что концентрация группы 0,75-1 и общее колличество точек выше в уберКлассах 2, 4.
Схопнул таблицу до уберКлассов 2, 4 и квантилей 0,75-1

ПараметрTotal в новой таблице разбил по группам квантилей 0-25, 25-50, 50-75, 75-100.

#### Работа с графиком распределения:
Высокая концентрация 0,75-1 у класса №3.
Затем классы 2 (привалирует 0,25-0,5 над 0,5-0,75) и 1 (низкие 0,75-1 и 0,25-0,5, средние 0,5-0,75)

Схлопнул до уберКлассов 3, 2.

#### Работа с матрицей корреляции:
Появился положительный коэф корреляции переменной Class с:
Total - 0.32
Attack - 0.27
Defence - 0.2
Generation - 0.19
Legendary - 0.29


#### На основе графиков парных взаимосвязей Class с разбивкой по группам Total_Groups:

1) Total - Явная положительная зависимость.

2) Legendary - Явная положительная зависимость.

3) Attack - Явная положительная зависимость.

## На основе анализа:
Для увеличения количества побед на соревнованиях покемонов необходимо сконцентрировать внимание на существах Типа: Dragon, при их получении на сучещствах Типа: Psychic согласно таблицам: Top_10_Dragon, Top_10_Psychic